<a href="https://colab.research.google.com/github/ArtyomGrachev/defeater/blob/ArtyomGrachev-PointNet-DeepCosineMetric-v1/PointNet_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creates pytorch-style Waymo-OD dataset for 3D point classification

In [0]:
!pip install --upgrade pip
!apt-get install tree
!pip install importlib
!pip install torchvision==0.4.0
!pip install  Pillow==6.2.1

In [0]:
# Clone github Waymo-od repo with util functions

!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!pip install waymo-open-dataset

Cloning into 'waymo-od'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 700 (delta 34), reused 39 (delta 14), pack-reused 618
Receiving objects: 100% (700/700), 14.17 MiB | 34.22 MiB/s, done.
Resolving deltas: 100% (417/417), done.
     |████████████████████████████████| 2.7 MB 4.9 MB/s 


In [0]:
# required libs

import sys
import os
import shutil
import inspect
import tarfile
import tqdm

import random
import math
import itertools
import functools
import collections

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.utils.data as tdata
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets 

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset.utils import  box_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [0]:
# Uncomment to download train segment from Waymo google cloud into google drive
# Training sets: training_0000.tar - training_0031.tar
# Validation sets: validation_0000.tar - validation_0007.tar

#!gsutil cp gs://waymo_open_dataset_v_1_0_0/training/training_0001.tar /content/drive/'My Drive'/Waymo_OD/

def extract_from_tar(extract_from, extract_to):
  file_tar = tarfile.open(name=extract_from, mode='r', fileobj=None, bufsize=10240)"
  file_tar.extractall(path=extract_to)


#os.remove(r"/content/drive/My Drive/Waymo_OD/training_0001.tar")
#os.remove(r"/content/drive/My Drive/Waymo_OD/LICENSE")

In [0]:
class PointCloudParser(object):
  """Performs point cloud 3D-box filtering for a given set of classes."""

  def __init__(self, data_path, max_segments=None, cls_to_filter=("TYPE_PEDESTRIAN", "TYPE_VEHICLE")):
    """
    data_path: Data path to folder that containts Waymo TFRecords

    max_segments: number of segments to process (None eq. all segments)

    cls_to_filter: point clouds for these classes will be extracted
    """

    assert os.path.exists(data_path), "Existing path is required"
    assert np.all([os.path.splitext(file_name)[-1] == ".tfrecord" 
                   for file_name in os.listdir(data_path)]), "All files in the folder should be .tfrecord"

    self.data_path = data_path

    self.cls_to_filter = cls_to_filter

    self.segments_to_proc = os.listdir(self.data_path)[:max_segments]

    self.type_enum = {
                        "TYPE_UNKNOWN" : 0,
                        "TYPE_VEHICLE" : 1,
                        "TYPE_PEDESTRIAN" : 2,
                        "TYPE_SIGN" : 3,
                        "TYPE_CYCLIST" : 4
                     }

    self.code_2_type = {v:k for k,v in self.type_enum.items()}

    self.obj_codes = tuple(self.type_enum[ctf] for ctf in self.cls_to_filter)
  
    self.bbox_count_dict = collections.defaultdict(int)

  def start_processing(self, dest_path):
    """
    Main routine method that starts points filtering. 
    Creates len(self.obj_codes) number folders in dest_path 
    for each class and saves point clouds for each box in .npy binary file. 
    """

    self.check_dir(dest_path)
    self.dest_path = dest_path

    for segment_name in self.segments_to_proc:
      dataset = tf.data.TFRecordDataset(
          os.path.join(self.data_path, segment_name), compression_type="")
      self.filter_frame_points(dataset)
      

  def filter_frame_points(self, dataset, min_pts_threshold=75):
    """
    Return dict that maps from object type ids to point clouds. 
    Value for each key is a list of numpy arrays, 
    where each numpy array containts points for a single 3D bbox.
    """

    bbox_dict = {obj:[] for obj in self.obj_codes}

    for data in tqdm.tqdm_notebook(dataset):
      frame = open_dataset.Frame()
      
      frame.ParseFromString(bytearray(data.numpy()))

      frame_pts = self.frame_points(frame)

      for obj_code in self.obj_codes:
        boxes = self.frame_boxes_3D(frame, obj_code)

        if len(boxes):
          bbox_dict[obj_code].extend(
              
               [
                filt_pts for filt_pts in (tf.boolean_mask(frame_pts, mask).numpy() 
                                          for mask in tf.transpose(
                                              box_utils.is_within_box_3d(frame_pts, boxes)
                                          )) 
                if len(filt_pts) >= min_pts_threshold
               ]
          )
    
    self.save_point_cloud(bbox_dict)
    

  def frame_points(self, frame, projection=False):
    """
    Returns Tensor of points in a vehicle coord. system for a given frame. 
    If projection is True => returns points projection on frame
    """
    (range_images, camera_projections, range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)

    points, cp_points = frame_utils.convert_range_image_to_point_cloud(
        frame,
        range_images,
        camera_projections,
        range_image_top_pose,
        ri_index=0)

    if projection:
      return tf.constant(np.concatenate(cp_points))
    else:
      return tf.constant(np.concatenate(points, axis=0))


  def frame_boxes_3D(self, frame, obj_code):
    """
    Returns np.array of box descriptors for a given frame
    """

    box_stats = np.array([tf.constant([lable.box.center_x, lable.box.center_y, lable.box.center_z, 
                  lable.box.width, lable.box.length, lable.box.height, lable.box.heading])
                  for lable in frame.laser_labels if lable.type == obj_code])

    return box_stats


  def check_dir(self, dest_path):
    """Creates required dir. for files"""
    if not os.path.exists(dest_path):
      os.mkdir(dest_path)
    for obj_code in self.obj_codes:
      obj_folder = os.path.join(dest_path, self.code_2_type[obj_code])
      if not os.path.exists(obj_folder):
        os.mkdir(obj_folder)


  def save_point_cloud(self, bbox_dict):
    """Saves filtered point cloud (numpy array of size [N, 3]) into .npy binary file."""

    print("Saving.")
    
    for obj_code in bbox_dict.keys():
      for bbox in bbox_dict[obj_code]:
        np.save(
                  os.path.join(self.dest_path, self.code_2_type[obj_code], 
                              "{0}.npy".format(self.bbox_count_dict[obj_code]).zfill(12)), 
                  bbox
               )
        self.bbox_count_dict[obj_code] += 1
        





In [0]:
# Point cloud processing functions
# segments_path = "/content/drive/My Drive/Waymo_OD/"
# testing_pcp = PointCloudParser(segments_path)
# testing_pcp.start_processing("/content/drive/My Drive/PointNet/Data_folder_train_0001")

In [0]:
# shutil.rmtree("/content/drive/My Drive/PointNet/Data_folder_train_0001")

In [0]:
def box_metadata_parser(dataset, dataset_id, obj_type="TYPE_PEDESTRIAN", save=False):
  """
  Compute box metadata from a given segment.
  Box metadata includes box track_id and L2 distance to box in vehicle coord. system
  """
  df_dict_template = {"track_id" : [], "box_dist" : []}

  type_enum = {
      "TYPE_UNKNOWN" : 0,
      "TYPE_VEHICLE" : 1,
      "TYPE_PEDESTRIAN" : 2,
      "TYPE_SIGN" : 3,
      "TYPE_CYCLIST" : 4,
    }

  obj_code = type_enum[obj_type]

  for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    for lable in frame.laser_labels:
      if lable.type == obj_code:
        df_dict_template["track_id"].append(lable.id)
        df_dict_template["box_dist"].append(np.linalg.norm([lable.box.center_x, lable.box.center_y, lable.box.center_z]))

  metadata_df = pd.DataFrame.from_dict(df_dict_template)

  if save:
    if not os.path.exists("./box_metadata"):
      os.mkdir("./box_metadata")
    
    metadata_df.to_csv(".//box_metadata//metadata_{0}_{1}.csv".format(obj_type, dataset_id), index = False, header=True)

  return metadata_df


### Create data loaders for train and validation


In [0]:
class MinPointSampler(object):
  """Transfromation that samples pts_num number of points from the input point cloud"""
  def __init__(self, pts_num):
    self.pts_num = pts_num

  def __call__(self, point_cloud):
    return point_cloud[np.random.choice(point_cloud.shape[0], size=self.pts_num, replace=False), :]


class PointScaler(object):
  """Scales point cloud to a new one with mean = 0 and maximum vector length of 1"""
  def __call__(self, point_cloud): 
    return (point_cloud - np.mean(point_cloud, axis=0)) / np.linalg.norm(point_cloud, axis=1).max()


class RndPointsAugmentations(object):
  """
  Performs 2 types of point data augmentation: 
  Random jittering via uniformly distributed noise and random rotation along z-axis
  """
  def __init__(self, jitter_a=0, jitter_b=0.2):
    self.jitter_a = jitter_a
    self.jitter_b = jitter_b
    
  def __call__(self, point_cloud):
    theta = np.random.uniform(0, np.pi*2)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    point_cloud[:, :2] = point_cloud[:, :2].dot(rotation_matrix)
    return point_cloud + np.random.normal(self.jitter_a, self.jitter_b, size=point_cloud.shape)


def create_train_val_data_loaders(data_dir, *, min_pts=75, batch_size=32, validation_frac = 0.2, num_of_workers=0):
    """
    Return pair of pytorch dataloaders for train and validation sets.
    """
    # sample => scale => (if train) random jitter and random rotation along z axis => transform to Pytorch tensor 

    mps_transform = MinPointSampler(min_pts)
    pt_scaler = PointScaler()
    points_aug = RndPointsAugmentations()

    train_transforms = transforms.Compose([
                                           mps_transform,
                                           points_aug,
                                           pt_scaler,
                                           transforms.ToTensor()
                                          ])
    
    val_transforms = transforms.Compose([
                                        mps_transform,
                                        pt_scaler,
                                        transforms.ToTensor()
                                       ])
    
    train_data = datasets.DatasetFolder(data_dir, loader=np.load, extensions=("npy"), 
                                        transform=train_transforms)

    val_data = datasets.DatasetFolder(data_dir, loader=np.load, extensions=("npy"),
                                      transform=val_transforms)
        
    dataset_len = len(train_data)

    indices = np.arange(dataset_len)
    
    val_abs_size = np.int(np.floor(validation_frac * dataset_len))

    np.random.shuffle(indices)
    
    train_id, val_id = indices[val_abs_size:], indices[:val_abs_size]
    
    train_sampler = tdata.SubsetRandomSampler(train_id)
    
    val_sampler = tdata.SubsetRandomSampler(val_id)
    
    train_loader = tdata.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, num_workers=num_of_workers)
    
    val_loader = tdata.DataLoader(val_data,
                   sampler=val_sampler, batch_size=batch_size, num_workers=num_of_workers)
    
    return train_loader, val_loader

### PointNet

In [0]:
class STN3d(nn.Module):
  """TNet for 3D point transformation"""
  def __init__(self):
      super(STN3d, self).__init__()
      self.conv1 = torch.nn.Conv1d(3, 64, 1)
      self.conv2 = torch.nn.Conv1d(64, 128, 1)
      self.conv3 = torch.nn.Conv1d(128, 1024, 1)
      self.fc1 = nn.Linear(1024, 512)
      self.fc2 = nn.Linear(512, 256)
      self.fc3 = nn.Linear(256, 9)
      self.relu = nn.ReLU()

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)


  def forward(self, x):
      batchsize = x.size()[0]
      x = F.relu(self.bn1(self.conv1(x)))
      x = F.relu(self.bn2(self.conv2(x)))
      x = F.relu(self.bn3(self.conv3(x)))
      x = torch.max(x, 2, keepdim=True)[0]
      x = x.view(-1, 1024)

      x = F.relu(self.bn4(self.fc1(x)))
      x = F.relu(self.bn5(self.fc2(x)))
      x = self.fc3(x)

      iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
      if x.is_cuda:
          iden = iden.cuda()
      x = x + iden
      x = x.view(-1, 3, 3)
      return x


class STNkd(nn.Module):
  """Tnet for k-D point transformation"""
  def __init__(self, k=64):
      super(STNkd, self).__init__()
      self.conv1 = torch.nn.Conv1d(k, 64, 1)
      self.conv2 = torch.nn.Conv1d(64, 128, 1)
      self.conv3 = torch.nn.Conv1d(128, 1024, 1)
      self.fc1 = nn.Linear(1024, 512)
      self.fc2 = nn.Linear(512, 256)
      self.fc3 = nn.Linear(256, k*k)
      self.relu = nn.ReLU()

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)

      self.k = k

  def forward(self, x):
      batchsize = x.size()[0]
      x = F.relu(self.bn1(self.conv1(x)))
      x = F.relu(self.bn2(self.conv2(x)))
      x = F.relu(self.bn3(self.conv3(x)))
      x = torch.max(x, 2, keepdim=True)[0]
      x = x.view(-1, 1024)

      x = F.relu(self.bn4(self.fc1(x)))
      x = F.relu(self.bn5(self.fc2(x)))
      x = self.fc3(x)

      iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
      if x.is_cuda:
          iden = iden.cuda()
      x = x + iden
      x = x.view(-1, self.k, self.k)
      return x

class PointNetfeat(nn.Module):
  """PointNet features part"""
  def __init__(self, global_feat = True, feature_transform = False):
      super(PointNetfeat, self).__init__()
      self.stn = STN3d()
      self.conv1 = torch.nn.Conv1d(3, 64, 1)
      self.conv2 = torch.nn.Conv1d(64, 128, 1)
      self.conv3 = torch.nn.Conv1d(128, 1024, 1)
      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.global_feat = global_feat
      self.feature_transform = feature_transform
      if self.feature_transform:
          self.fstn = STNkd(k=64)

  def forward(self, x):
      n_pts = x.size()[2]
      trans = self.stn(x)
      x = x.transpose(2, 1)
      x = torch.bmm(x, trans)
      x = x.transpose(2, 1)
      x = F.relu(self.bn1(self.conv1(x)))

      if self.feature_transform:
          trans_feat = self.fstn(x)
          x = x.transpose(2,1)
          x = torch.bmm(x, trans_feat)
          x = x.transpose(2,1)
      else:
          trans_feat = None

      pointfeat = x
      x = F.relu(self.bn2(self.conv2(x)))
      x = self.bn3(self.conv3(x))
      x = torch.max(x, 2, keepdim=True)[0]
      x = x.view(-1, 1024)
      if self.global_feat:
          return x, trans, trans_feat
      else:
          x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
          return torch.cat([x, pointfeat], 1), trans, trans_feat

class DeepCosineMetric(nn.Module):
  """Deep cosine metric net-end"""
  def __init__(self, k=2, scale_param=1):
    super(DeepCosineMetric, self).__init__()
    self.fc_last = nn.Linear(256, k, bias=False)
    self.scale_param = scale_param
  
  def forward(self, x):
    with torch.no_grad():
      self.fc_last.weight.div_(torch.norm(self.fc_last.weight, dim=1, keepdim=True))

    x = F.normalize(x, p=2, dim=1, eps=1e-12)

    x = self.scale_param*self.fc_last(x)

    return F.log_softmax(x, dim=1)



class PointNetCls(nn.Module):
  """PointNet for classification"""
  def __init__(self, k=2, feature_transform=False):
      super(PointNetCls, self).__init__()
      self.feature_transform = feature_transform
      self.feat = PointNetfeat(global_feat=True, feature_transform=feature_transform)
      self.fc1 = nn.Linear(1024, 512)
      self.fc2 = nn.Linear(512, 256)
      self.deep_cosine_metric = DeepCosineMetric(k=k)

      self.dropout = nn.Dropout(p=0.3)
      self.bn1 = nn.BatchNorm1d(512)
      self.bn2 = nn.BatchNorm1d(256)
      self.relu = nn.ReLU()

  def forward(self, x):
      x, trans, trans_feat = self.feat(x)
      x = F.relu(self.bn1(self.fc1(x)))
      x = F.relu(self.bn2(self.dropout(self.fc2(x))))
      x = self.deep_cosine_metric(x)
      return x, trans, trans_feat



def feature_transform_regularizer(trans):
  """Regularization for TNet (TNet transformation matrix should be close to orthogonal)"""
  d = trans.size()[1]
  batchsize = trans.size()[0]
  I = torch.eye(d)[None, :, :]
  if trans.is_cuda:
    I = I.to("cuda")
  loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
  return loss

### Training

In [0]:
# Hyperparams

manual_seed = 424242                        # seed for rng
cls_num = 2                                 # number of classes, should match that number in dataloader 
pointnet_feature_transform = True           # Use T-Net block for inner features transformation
existing_model_path = ""                    # download model state from path ("" if no pretrained model to use)
epoch_number = 1                            # epoch to train  
batch_size = 32                             # should batch match batch size in dataloader
val_step = 10                               # evaluate network on validation set example if cur_step % val_step == 0
reg_lambda = 0.001                          # coefficient for T-Net orthogonal regularization
save_model_to = "/content/drive/My Drive/PointNet/Model/"    # path to folder where model will be saved

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

adam_parameters = {"lr": 0.001, "betas": (0.9, 0.999)}
shelduler_parameters = {"step_size": 20, "gamma": 0.5}

In [0]:
random.seed(manual_seed)
torch.manual_seed(manual_seed)

point_net_cls = PointNetCls(k=cls_num, feature_transform=pointnet_feature_transform)

train_loader, val_loader = create_train_val_data_loaders("/content/drive/My Drive/PointNet/Data_folder_train_0001/", 
                                                         num_of_workers=2)

if existing_model_path != "":
    point_net_cls.load_state_dict(torch.load(existing_model_path))

optimizer = optim.Adam(point_net_cls.parameters(), **adam_parameters)

scheduler = optim.lr_scheduler.StepLR(optimizer, **shelduler_parameters)

point_net_cls.cuda()

num_batches = len(train_loader)

for epoch in range(epoch_number):
    scheduler.step()

    for i, [points, target] in enumerate(train_loader):

        points = points.squeeze().transpose(2, 1)

        points, target = points.to(device, dtype=torch.float), target.to(device) # tensors to GPU

        optimizer.zero_grad()

        point_net_cls = point_net_cls.train()

        pred, trans, trans_feat = point_net_cls(points)

        loss = F.nll_loss(pred, target)
        
        if pointnet_feature_transform:
            loss += feature_transform_regularizer(trans_feat) * reg_lambda

        loss.backward()

        optimizer.step()

        pred_choice = pred.data.max(1)[1]

        correct = pred_choice.eq(target.data).cpu().sum()

        print('[epoch #{0}: {1}/{2}] train loss: {3} batch accuracy: {4}'.format(epoch, i, num_batches, loss.item(), correct.item() / float(batch_size)))

        if i % val_step == 0:
            j, [points, target] = next(enumerate(val_loader))

            points = points.squeeze().transpose(2, 1)

            points, target = points.cuda(), target.cuda()

            point_net_cls = point_net_cls.eval()

            pred, _, _ = point_net_cls(points)

            loss = F.nll_loss(pred, target)

            pred_choice = pred.data.max(1)[1]

            correct = pred_choice.eq(target.data).cpu().sum()

            print('[epoch #{0}: {1}/{2}] validation loss: {3} batch accuracy: {4}'.format(epoch, i, num_batches, loss.item(), correct.item()/float(batch_size)))

        torch.save(point_net_cls.state_dict(), os.path.join(save_model_to, "cls_model_{0}.pth".format(epoch)))



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[epoch #0: 0/502] train loss: 0.8487129211425781 batch accuracy: 0.375
[epoch #0: 0/502] validation loss: 0.8049327731132507 batch accuracy: 0.25
[epoch #0: 1/502] train loss: 0.7864579558372498 batch accuracy: 0.625
[epoch #0: 2/502] train loss: 0.7720398306846619 batch accuracy: 0.625
[epoch #0: 3/502] train loss: 0.7167998552322388 batch accuracy: 0.71875
[epoch #0: 4/502] train loss: 0.7652473449707031 batch accuracy: 0.65625
[epoch #0: 5/502] train loss: 0.7493324279785156 batch accuracy: 0.625
[epoch #0: 6/502] train loss: 0.7548136115074158 batch accuracy: 0.65625
[epoch #0: 7/502] train loss: 0.6900687217712402 batch accuracy: 0.8125
[epoch #0: 8/502] train loss: 0.7332444190979004 batch accuracy: 0.75
[epoch #0: 9/502] train loss: 0.6733198165893555 batch accuracy: 0.75
[epoch #0: 10/502] train loss: 0.7535916566848755 batch accuracy: 0.65625
[epoch #0: 10/502] validation loss: 0.6300651431083679 batch accuracy: 0.78125
[epoch #0: 11/502] train loss: 0.7039129137992859 batch a